In [21]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

from src.logger import logging

In [22]:
from winotify import Notification, audio

toast = Notification(app_id='365 Learning Data Challenge',
                     title='merged_info_purchase',
                     msg='Code execution done.')

toast.set_audio(audio.Mail, loop=False)

In [23]:
course_info_df = pd.read_csv('data/raw/365_course_info.csv')
course_ratings_df = pd.read_csv('data/raw/365_course_ratings.csv')
exam_info_df = pd.read_csv('data/raw/365_exam_info.csv')
quiz_info_df = pd.read_csv('data/raw/365_quiz_info.csv')
student_engagement_df = pd.read_csv('data/raw/365_student_engagement.csv')
student_exams_df = pd.read_csv('data/raw/365_student_exams.csv')
student_hub_questions_df = pd.read_csv('data/raw/365_student_hub_questions.csv')
student_info_df = pd.read_csv('data/raw/365_student_info.csv')
student_learning_df = pd.read_csv('data/raw/365_student_learning.csv')
student_purchases_df = pd.read_csv('data/raw/365_student_purchases.csv')
student_quizzes_df = pd.read_csv('data/raw/365_student_quizzes.csv')

In [24]:
course_ratings_df.date_rated = pd.to_datetime(course_ratings_df.date_rated)
student_engagement_df.date_engaged = pd.to_datetime(student_engagement_df.date_engaged)
student_exams_df.date_exam_completed = pd.to_datetime(student_exams_df.date_exam_completed)
student_hub_questions_df.date_question_asked = pd.to_datetime(student_hub_questions_df.date_question_asked)
student_info_df.date_registered = pd.to_datetime(student_info_df.date_registered)
student_learning_df.date_watched = pd.to_datetime(student_learning_df.date_watched)
student_purchases_df.date_purchased = pd.to_datetime(student_purchases_df.date_purchased)

In [25]:
student_learning_minutes_watched = student_learning_df[['student_id', 'minutes_watched']].groupby('student_id').sum().reset_index()

merged_student_learning_engagement = pd.merge(student_learning_minutes_watched, student_engagement_df, on='student_id')
merged_student_learning_engagement = merged_student_learning_engagement.drop(['date_engaged', 'engagement_id'], axis=1)
merged_student_learning_engagement = merged_student_learning_engagement.groupby(['student_id', 'minutes_watched']).agg({'engagement_quizzes': np.sum, 'engagement_exams': np.sum, 'engagement_lessons': np.sum}).reset_index()

merged_student_learning_engagement

,student_id,minutes_watched,engagement_quizzes,engagement_exams,engagement_lessons
0,258798,0.3,0,0,1
1,258800,531.2,10,5,29
2,258803,620.0,7,4,19
3,258804,14.2,1,0,1
4,258807,1.8,0,0,1
...,...,...,...,...,...
18159,295509,0.8,0,0,1
18160,295510,0.1,0,0,1
18161,295511,7.8,1,0,1
18162,295512,10.4,1,0,1


In [26]:
merged_info_purchase = pd.merge(student_info_df, student_purchases_df, on='student_id', how='left')
merged_info_purchase = merged_info_purchase.drop_duplicates(subset='student_id', keep='last')

In [52]:
def is_paid_tier(student_id: str):
    '''
    returns: [active_purchase, ]
    '''

    if student_id not in student_purchases_df.values:
        return False, False

    df = merged_info_purchase[merged_info_purchase.student_id == student_id]

    latest_date = datetime(2022, 10, 20).date()
    latest_date_purchased = df.date_purchased.iloc[0].date()

    latest_purchase_type = df.purchase_type.iloc[0]

    difference = (latest_date - latest_date_purchased).days

    if latest_purchase_type == 'Monthly':
        if difference <= 30: return True, True
        if 30 < difference <= 30 + 90: return False, True
        if difference > 30 + 90: return False, False
    
    if latest_purchase_type == 'Quarterly':
        if difference <= 90: return True, True
        if 30 < difference <= 90 + 90: return False, True
        if difference > 90 + 90: return False, False
    
    if latest_purchase_type == 'Annual':
        if difference <= 365: return True, True
        if 30 < difference <= 365 + 90: return False, True
        if difference > 365 + 90: return False, False

    # if latest_purchase_type == 'Monthly':
    #     return pd.Series([difference.days <= 30])
    # if latest_purchase_type == 'Quarterly':
    #     return pd.Series([difference.days <= 90])
    # if latest_purchase_type == 'Annual':
    #     return pd.Series([difference.days <= 365])

In [53]:
merged_info_purchase['paid'] = merged_info_purchase.student_id.apply(lambda id : is_paid_tier(id))
toast.show()

In [54]:
merged_info_purchase

,student_id,student_country,date_registered,purchase_id,purchase_type,date_purchased,paid
0,258798,IN,2022-01-01,NaN,NaN,NaT,"(False, False)"
1,258799,CO,2022-01-01,NaN,NaN,NaT,"(False, False)"
2,258800,CA,2022-01-01,15781.0,Annual,2022-01-01,"(True, True)"
3,258801,IN,2022-01-01,NaN,NaN,NaT,"(False, False)"
4,258802,US,2022-01-01,NaN,NaN,NaT,"(False, False)"
...,...,...,...,...,...,...,...
36131,295511,CM,2022-10-20,NaN,NaN,NaT,"(False, False)"
36132,295512,CA,2022-10-20,NaN,NaN,NaT,"(False, False)"
36133,295513,SA,2022-10-20,NaN,NaN,NaT,"(False, False)"
36134,295514,NL,2022-10-20,NaN,NaN,NaT,"(False, False)"


In [58]:
merged_info_purchase['active_purchase'] = merged_info_purchase.paid.apply(lambda list : list[0])
merged_info_purchase['potential_stay'] = merged_info_purchase.paid.apply(lambda list : list[1])
merged_info_purchase[merged_info_purchase.purchase_type == 'Monthly']

,student_id,student_country,date_registered,purchase_id,purchase_type,date_purchased,paid,potential_stay,active_purchase
23,258822,IN,2022-01-01,15941.0,Monthly,2022-01-09,"(False, False)",False,False
33,258832,IN,2022-01-01,17437.0,Monthly,2022-03-05,"(False, False)",False,False
107,258908,US,2022-01-01,15813.0,Monthly,2022-01-01,"(False, False)",False,False
157,258956,IN,2022-01-02,20476.0,Monthly,2022-06-30,"(False, True)",True,False
172,258971,IL,2022-01-02,15956.0,Monthly,2022-01-09,"(False, False)",False,False
...,...,...,...,...,...,...,...,...,...
35579,294936,IN,2022-10-16,23080.0,Monthly,2022-10-17,"(True, True)",True,True
35728,295088,NL,2022-10-17,23096.0,Monthly,2022-10-18,"(True, True)",True,True
35891,295256,CA,2022-10-19,23106.0,Monthly,2022-10-19,"(True, True)",True,True
35908,295273,TR,2022-10-19,23111.0,Monthly,2022-10-19,"(True, True)",True,True
